In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import classification_report
from tensorflow.keras import layers
from keras.constraints import maxnorm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation,  Flatten, Input
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras import metrics
from tensorflow import keras
import joblib

In [2]:
# 学習データの読み込み
train_df=pd.read_csv("/Users/tasuke0630/making/DataScience/kaggle/Taitanic/train.csv")

# テストデータの読み込み
test_df=pd.read_csv("/Users/tasuke0630/making/DataScience/kaggle/Taitanic/test.csv")

# 学習データとテストデータを連結する
df=pd.concat([train_df,test_df],ignore_index=True)
#nullのCSV化
df_null=df.isnull().sum()
df_null.to_csv("null.csv")

# 元データをコピー
df2=df.copy()

In [3]:
#null_csvを見ながら,nullのデータの確認
#sns.countplot("Survived",data=df2)
#sns.countplot("Age",data=df2)
#sns.countplot("Fare",data=df2)
#sns.countplot("Cabin",data=df2)
#sns.countplot("Embarked",data=df2)

In [4]:
print(df2.Cabin.mode())
print(df2.Fare.mode())

Age_mdeian=df2.Age.median()
df2.Age=df2.Age.fillna(Age_mdeian)

0    C23 C25 C27
dtype: object
0    8.05
dtype: float64


In [5]:
print(df2.Cabin.mode())
print(df2.Fare.mode())

Age_mdeian=df2.Age.median()
df2.Age=df2.Age.fillna(Age_mdeian)

0    C23 C25 C27
dtype: object
0    8.05
dtype: float64


In [6]:
print(df2.Cabin.mode())
print(df2.Fare.mode())

Age_mdeian=df2.Age.median()
df2.Age=df2.Age.fillna(Age_mdeian)

0    C23 C25 C27
dtype: object
0    8.05
dtype: float64


In [7]:
# 欠損値の補完
df2.Fare=df2.Fare.fillna(8.05)
df2.Cabin=df2.Cabin.fillna("C23")
df2.Embarked=df2.Embarked.fillna("S")

In [8]:
df2.isnull().sum()

PassengerId      0
Survived       418
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin            0
Embarked         0
dtype: int64

In [9]:
#使わないカラムを削除する
df3=df2.drop(columns=["Name","Ticket"])
df3

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,1,0.0,3,male,22.0,1,0,7.2500,C23,S
1,2,1.0,1,female,38.0,1,0,71.2833,C85,C
2,3,1.0,3,female,26.0,0,0,7.9250,C23,S
3,4,1.0,1,female,35.0,1,0,53.1000,C123,S
4,5,0.0,3,male,35.0,0,0,8.0500,C23,S
...,...,...,...,...,...,...,...,...,...,...
1304,1305,NaN,3,male,28.0,0,0,8.0500,C23,S
1305,1306,NaN,1,female,39.0,0,0,108.9000,C105,C
1306,1307,NaN,3,male,38.5,0,0,7.2500,C23,S
1307,1308,NaN,3,male,28.0,0,0,8.0500,C23,S


In [10]:
#カテゴリカル変数の数値変換
tmp_Sex=pd.get_dummies(df3["Sex"],prefix="Sex")
df3=pd.concat([df3,tmp_Sex],axis=1).drop(columns="Sex")

tmp_Cabin=pd.get_dummies(df3["Cabin"],prefix="Cabin")
df3=pd.concat([df3,tmp_Cabin],axis=1).drop(columns="Cabin")

tmp_Embarked=pd.get_dummies(df3["Embarked"],prefix="Embarked")
df3=pd.concat([df3,tmp_Embarked],axis=1).drop(columns="Embarked")
df3.sample(10)

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Cabin_A10,...,Cabin_F G73,Cabin_F2,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T,Embarked_C,Embarked_Q,Embarked_S
452,453,0.0,1,30.0,0,0,27.7500,0,1,0,...,0,0,0,0,0,0,0,1,0,0
382,383,0.0,3,32.0,0,0,7.9250,0,1,0,...,0,0,0,0,0,0,0,0,0,1
449,450,1.0,1,52.0,0,0,30.5000,0,1,0,...,0,0,0,0,0,0,0,0,0,1
813,814,0.0,3,6.0,4,2,31.2750,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1295,1296,NaN,1,43.0,1,0,27.7208,0,1,0,...,0,0,0,0,0,0,0,1,0,0
373,374,0.0,1,22.0,0,0,135.6333,0,1,0,...,0,0,0,0,0,0,0,1,0,0
1222,1223,NaN,1,39.0,0,0,29.7000,0,1,0,...,0,0,0,0,0,0,0,1,0,0
761,762,0.0,3,41.0,0,0,7.1250,0,1,0,...,0,0,0,0,0,0,0,0,0,1
459,460,0.0,3,28.0,0,0,7.7500,0,1,0,...,0,0,0,0,0,0,0,0,1,0
118,119,0.0,1,24.0,0,1,247.5208,0,1,0,...,0,0,0,0,0,0,0,1,0,0


In [11]:
# 学習データに分割した結果を変数trainに格納する
train=df3[~df3.Survived.isnull()]

# テストデータに分割した結果を変数trainに格納する
test=df3[df3.Survived.isnull()]

#trainのデータ
train_x=train.drop(columns=["Survived","PassengerId"])
train_y=train["Survived"]

In [13]:
def making_model():
    model=Sequential()
    model.add(Dense(9,input_dim=197,activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(11,activation="relu"))
    model.add(Flatten())
    optimizer=keras.optimizers.RMSprop(lr=0.008)
    model.add(Dense(units=1,activation="sigmoid"))
    model.compile(loss="binary_crossentropy",
                  optimizer=optimizer,metrics=[keras.metrics.AUC(),"accuracy"])
    return model


KerasC=KerasClassifier(build_fn=making_model,epochs=50,batch_size= 30)
KerasC._estimator_type ="classifier"

KerasC.fit(train_x, train_y)

Y_pred_train=KerasC.predict(train_x)
print( classification_report(train_y,Y_pred_train))

X_test=test.drop(columns=["Survived","PassengerId"])
y_predict= KerasC.predict(X_test)

print("Stacking model score: %.3f" % KerasC.score(X_test, y_predict))

Epoch 1/50
30/30 [==============================] - 1s 1ms/step - loss: 0.8910 - auc_1: 0.6484 - accuracy: 0.6131
Epoch 2/50
30/30 [==============================] - 0s 2ms/step - loss: 0.6417 - auc_1: 0.6736 - accuracy: 0.6522
Epoch 3/50
30/30 [==============================] - 0s 1ms/step - loss: 0.5763 - auc_1: 0.7366 - accuracy: 0.7047
Epoch 4/50
30/30 [==============================] - 0s 1ms/step - loss: 0.5726 - auc_1: 0.7619 - accuracy: 0.7070
Epoch 5/50
30/30 [==============================] - 0s 2ms/step - loss: 0.5457 - auc_1: 0.7724 - accuracy: 0.7510
Epoch 6/50
30/30 [==============================] - 0s 1ms/step - loss: 0.5403 - auc_1: 0.7954 - accuracy: 0.7585
Epoch 7/50
30/30 [==============================] - 0s 2ms/step - loss: 0.5300 - auc_1: 0.8009 - accuracy: 0.7665
Epoch 8/50
30/30 [==============================] - 0s 1ms/step - loss: 0.5009 - auc_1: 0.8222 - accuracy: 0.7757
Epoch 9/50
30/30 [==============================] - 0s 2ms/step - loss: 0.4863 - auc_1: 

/Users/tasuke0630/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
/Users/tasuke0630/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model doe

14/14 [==============================] - 0s 1ms/step - loss: 0.2156 - auc_1: 1.0000 - accuracy: 1.0000
Stacking model score: 1.000


In [14]:
result=pd.DataFrame(y_predict)

#列名に名前をつける
result2=result.rename(columns={result.columns[0]: "Survived"})

#PassngerIdを合わせる
l=[]
l2=[]
t_id=test_df.loc[:,"PassengerId"]
for i in range(len(t_id)):
               tmp={
                   "PassengerId":t_id[i]
               }
               l.append(tmp);
               
l2=pd.DataFrame(l);
result3=pd.concat([l2,result2],axis=1)


#予測結果を整数に変換する
result3.Survived=result3.Survived.astype(int)
result3.PassengerId=result3.PassengerId.astype(int)

In [15]:
# CSVの作成
result3.to_csv("predict8.csv",index=False)